In [7]:
# this is a test script for a watson-integrated parts miner
# 
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, ConceptsOptions

authenticator = IAMAuthenticator('icmDJw0O-dSqYOoEcKwt-8j3uDsp22ULVfVFN_3fczSr')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2020-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.us-east.natural-language-understanding.watson.cloud.ibm.com/instances/b5ab2432-7e17-407e-a70a-2f10076a5ae4')






NameError: name 'filtered_corpus' is not defined

In [35]:
# read in practice corpus
import os
import os.path
practice_corpus = []
os.chdir('/Users/cullenpaulisick/Documents/EC552/Project/EC552SaturnLab/LDACorpus')
for filename in os.listdir(os.getcwd()):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='ISO-8859-1') as f:
        practice_corpus.append(f.read())
    f.close()
    

In [36]:

docs = list(practice_corpus)
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + ['fig', 'supplementary']
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 2] for doc in docs]
# Remove stop words
docs = [[token for token in doc if token not in stop_words] for doc in docs]
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
filtered_corpus = [" ".join(doc) for doc in docs]

In [12]:
lemma_dir = '/Users/cullenpaulisick/Documents/EC552/Project/EC552SaturnLab/WorkingCorpus/LemmaCorpus'
os.chdir(lemma_dir)

In [37]:
for i, text in enumerate(filtered_corpus[:50]):
    if len(text) >= 40000:
        filtered_corpus[i] = text[:39998]

In [39]:
for textfile in filtered_corpus[:2]:
    response = natural_language_understanding.analyze(
        text=textfile,
        features=Features(entities=EntitiesOptions(limit=10, model='116b1006-b509-463c-b13e-a57f7a6749f3'))
        ).get_result()



{
  "usage": {
    "text_units": 2,
    "text_characters": 18418,
    "features": 1
  },
  "language": "en",
  "entities": [
    {
      "type": "Part",
      "text": "eyfp",
      "disambiguation": {
        "subtype": [
          "Protein"
        ]
      },
      "count": 1,
      "confidence": 0.913608
    },
    {
      "type": "Part",
      "text": "sfgfp",
      "disambiguation": {
        "subtype": [
          "Protein"
        ]
      },
      "count": 1,
      "confidence": 0.802126
    },
    {
      "type": "Part",
      "text": "afb2",
      "disambiguation": {
        "subtype": [
          "Gene"
        ]
      },
      "count": 1,
      "confidence": 0.75608
    },
    {
      "type": "Part",
      "text": "afb",
      "disambiguation": {
        "subtype": [
          "Gene"
        ]
      },
      "count": 1,
      "confidence": 0.664737
    },
    {
      "type": "Method",
      "text": "turbidostats",
      "disambiguation": {
        "subtype": [
          "NONE

In [47]:
from json_flatten import flatten
import numpy as np
flat_response =flatten(response['entities'])
parts_list = [value for key, value in flat_response.items() if 'text' in key.lower()]
list(np.unique(parts_list))


['1ztu', 'bphp', 'ifp', 'ifp1', 'pdb', 'pgex2tk']

In [63]:
response['entities']

[{'type': 'Part',
  'text': 'ifp1',
  'disambiguation': {'subtype': ['Gene']},
  'count': 1,
  'confidence': 0.972359},
 {'type': 'Part',
  'text': 'ifp1',
  'disambiguation': {'subtype': ['Gene']},
  'count': 1,
  'confidence': 0.968187},
 {'type': 'Part',
  'text': 'pdb',
  'disambiguation': {'subtype': ['NONE']},
  'count': 1,
  'confidence': 0.963387},
 {'type': 'Part',
  'text': 'pgex2tk',
  'disambiguation': {'subtype': ['Promoter']},
  'count': 1,
  'confidence': 0.956383},
 {'type': 'Part',
  'text': '1ztu',
  'disambiguation': {'subtype': ['NONE']},
  'count': 1,
  'confidence': 0.955911},
 {'type': 'Part',
  'text': 'pgex2tk',
  'disambiguation': {'subtype': ['Promoter']},
  'count': 1,
  'confidence': 0.954264},
 {'type': 'Part',
  'text': 'ifp1',
  'disambiguation': {'subtype': ['Gene']},
  'count': 1,
  'confidence': 0.941114},
 {'type': 'Part',
  'text': 'pgex2tk',
  'disambiguation': {'subtype': ['Promoter']},
  'count': 1,
  'confidence': 0.900977},
 {'type': 'Part',
  

In [61]:
parts_list = []
gene_list = []
sequence_list = []
for item in response['entities']:
    if item['type'].lower() == 'part':
        if item['text'] not in parts_list:
            parts_list.append(item['text'])
            if item['disambiguation']['subtype'][0].lower() != 'none' and item['disambiguation']['subtype'][0].lower() != 'sequence' :
                gene_list.append(item['text'])
            if item['disambiguation']['subtype'][0].lower() == 'sequence':
                sequence_list.append(item['text'])
        


In [62]:
gene_list

['ifp1', 'pgex2tk']

In [67]:

from Bio import Entrez
results_dict = {}
for gene in gene_list:
    Entrez.email = 'clp0216@bu.edu'
    handle = Entrez.esearch(db='nucleotide',
                            sort='relevance',
                            retmax='810',
                            retmode='xml',
                            term=gene+"[gene]", 
                            idtype='acc')
    results_dict[gene] = Entrez.read(handle)




In [72]:
results_dict['ifp1']['IdList']

['568815587', '145275182', '51477686', '1677538733', '1677530040', '223461610', '187950470', '162317755', '11022691', '11022689', '11022687']

In [92]:
handle_set = []
for acc_num in results_dict['ifp1']['IdList'][:3]:
    fetch_handle = Entrez.efetch(db="nucleotide", id=acc_num, rettype="gb", retmode="text")
    data = fetch_handle.read()
    handle_set.append(data)

In [94]:
handle_set[0]

'LOCUS       NC_000011          135086622 bp    DNA     linear   CON 01-MAR-2021\nDEFINITION  Homo sapiens chromosome 11, GRCh38.p13 Primary Assembly.\nACCESSION   NC_000011\nVERSION     NC_000011.10\nDBLINK      BioProject: PRJNA168\n            Assembly: GCF_000001405.39\nKEYWORDS    RefSeq.\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1  (bases 1 to 135086622)\n  AUTHORS   Taylor,T.D., Noguchi,H., Totoki,Y., Toyoda,A., Kuroki,Y., Dewar,K.,\n            Lloyd,C., Itoh,T., Takeda,T., Kim,D.W., She,X., Barlow,K.F.,\n            Bloom,T., Bruford,E., Chang,J.L., Cuomo,C.A., Eichler,E.,\n            FitzGerald,M.G., Jaffe,D.B., LaButti,K., Nicol,R., Park,H.S.,\n            Seaman,C., Sougnez,C., Yang,X., Zimmer,A.R., Zody,M.C.,\n            Birren,B.W., Nusbaum,C., Fuj